Import

In [667]:
import pandas as pd
import re

Read CSV

In [668]:
df_bordeaux = pd.read_csv("CSV/bordeaux/listings_am.csv")


In [669]:
df_bordeaux.columns


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

### Question n°1  

In [670]:
df_bordeaux.groupby("neighbourhood_cleansed").apply(lambda s: pd.Series({ 
    "host count": s["host_id"].nunique(), 
    "sum reviews": s["number_of_reviews"].sum(), 
}))

,host count,sum reviews
neighbourhood_cleansed,,
Bijlmer-Centrum,76,4964
Bijlmer-Oost,29,983
Bos en Lommer,284,10586
Buitenveldert - Zuidas,111,5691
Centrum-Oost,489,45227
Centrum-West,674,69068
De Aker - Nieuw Sloten,69,6330
De Baarsjes - Oud-West,916,40372
De Pijp - Rivierenbuurt,644,30418


### Question n°2

In [671]:
df_host_acceptance_rate=df_bordeaux["host_acceptance_rate"].str.rstrip('%').astype(float)

df_host_acceptance_rate.mean()

80.17012104128669

In [672]:
df_host_response_rate=df_bordeaux["host_response_rate"].str.rstrip('%').astype(float)

df_host_response_rate.mean()

94.96734148204736

### Question n°3

In [673]:
df_bordeaux[['id',"host_verifications"]].groupby("host_verifications").count()/len(df_bordeaux)*100

,id
host_verifications,
"['email', 'phone', 'work_email']",12.737560
"['email', 'phone']",80.008704
['email'],0.072537
"['phone', 'work_email']",0.290149
['phone'],6.644422
[],0.246627


In [674]:
df_bordeaux.replace(to_replace="work_email", value='work', regex=True,inplace=True)

In [675]:
email = df_bordeaux["host_verifications"].apply(lambda elt: 1 if 'email' in elt else 0 ).sum()
phone = df_bordeaux["host_verifications"].apply(lambda elt: 1 if 'phone' in elt else 0 ).sum()
work = df_bordeaux["host_verifications"].apply(lambda elt: 1 if 'work' in elt else 0 ).sum()
vide = df_bordeaux["host_verifications"].apply(lambda elt: 1 if elt == "[]" else 0 ).sum()
  
print("Il y a {0} personnes qui utilise un email pour la vérification soit {1} %".format(email,email/len(df_bordeaux)*100))
print("Il y a {0} personnes qui utilise un téléphone pour la vérification soit {1} %".format(phone,phone/len(df_bordeaux)*100))
print("Il y a {0} personnes qui utilise un email pro pour la vérification soit {1} %".format(work,work/len(df_bordeaux)*100))
print("Il y a {0} personnes qui n'ont pas  pour la vérification soit {1} %".format(vide,vide/len(df_bordeaux)*100))

Il y a 6398 personnes qui utilise un email pour la vérification soit 92.81880168286668 %
Il y a 6871 personnes qui utilise un téléphone pour la vérification soit 99.68083563034963 %
Il y a 898 personnes qui utilise un email pro pour la vérification soit 13.02770927027419 %
Il y a 17 personnes qui n'ont pas  pour la vérification soit 0.24662701291164948 %


### Question n°4

In [676]:
df_bordeaux["np_amenities"] =  df_bordeaux['amenities'].apply(lambda elt: re.sub("\[|\]|\"","",elt))
df_bordeaux["np_amenities"] = df_bordeaux["np_amenities"].apply(lambda elt: elt.split(","))


In [677]:
df_bordeaux["np_amenities"] =  df_bordeaux["np_amenities"].apply(lambda x: len(x))


In [678]:
df_bordeaux[["np_amenities","room_type"]].groupby('room_type').agg(["mean","std"])

np_amenities           
                        mean        std
room_type                              
Entire home/apt    30.321617  14.252961
Hotel room         22.653333   9.515980
Private room       25.866057  12.292752
Shared room        13.675676   8.100254

### Question n° 5

In [679]:
df_bordeaux["price"]= df_bordeaux["price"].str.replace(',',"")
df_bordeaux["price"]= df_bordeaux["price"].str.replace('$',"").astype(float)

/tmp/ipykernel_6245/484732451.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_bordeaux["price"]= df_bordeaux["price"].str.replace('$',"").astype(float)


In [680]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [681]:
df_bordeaux[["price","room_type"]].groupby('room_type').agg(["median","max","min",q1,q3])

price                            
                median     max   min     q1     q3
room_type                                         
Entire home/apt  200.0  7900.0  15.0  143.0  285.0
Hotel room       222.0   999.0   0.0  114.5  271.0
Private room     130.0  1657.0  25.0   94.0  198.0
Shared room       75.0   500.0  45.0   66.0   94.0

### Question n°6

In [685]:
df_bordeaux["bathrooms_text"].unique()

array(['1.5 shared baths', '1 private bath', '1.5 baths', '1 shared bath',
       '1 bath', '2.5 baths', '3.5 baths', nan, '0 baths',
       'Private half-bath', '2 baths', '3 baths', '0 shared baths',
       'Half-bath', 'Shared half-bath', '5 baths', '2 shared baths',
       '5.5 baths', '4 baths', '4.5 baths', '4 shared baths',
       '3 shared baths', '17 baths', '3.5 shared baths'], dtype=object)